<a href="https://colab.research.google.com/github/OmdenaAI/SriLankaChapter_RegulatoryDecisionMaking/blob/main/task-4-model-building/experimental-notebooks/Explore_Optimal_Chunksize_RegulatoryRAG_allDocuments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
%pip install llama-index -q
%pip install transformers -q
%pip install torch -q
%pip install llama-index-llms-groq -q
%pip install sentence-transformers -q
%pip install "llama-index-embeddings-huggingface" -q
%pip install kdbai-client -q
%pip install llama-index-vector-stores-kdbai -q
%pip install kdbai_client -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2

In [2]:
import pandas as pd
from typing import List, Dict
from llama_index.core import VectorStoreIndex, ServiceContext, Document
from llama_index.core.node_parser import SentenceSplitter, MarkdownNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.core.llms import ChatMessage
import kdbai_client as kdbai

import time
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# Data Loading

In [3]:
def load_data(csv_path: str, text_col: List[str], metadata_cols: List[str]) -> List[Document]:
  """
  Load documents and include class in metadata
  """
  df = pd.read_csv(csv_path)
  df.fillna("nan", inplace=True)
  documents = []
  cols = ['original_doc_id', 'class', 'issuing_authority', 'title', 'issue_date', 'reference_number']
  for _, row in df.iterrows():
      text = str(row[text_col])
      doc = Document(
          text=text,
          metadata= {cols[i]: row[col] for i, col in enumerate(metadata_cols)}
      )
      documents.append(doc)
  return documents

DATA_PATH = "https://raw.githubusercontent.com/OmdenaAI/SriLankaChapter_RegulatoryDecisionMaking/main/data/final_dataset/v0_LK_tea_dataset.csv"
# DATA_PATH = "https://github.com/OmdenaAI/SriLankaChapter_RegulatoryDecisionMaking/blob/main/data/final_dataset/v0_LK_tea_dataset.csv"
text_col = 'markdown_content'
metadata_cols = ['id', 'class', 'issuing_authority', 'llama_title', 'llama_issue_date', 'llama_reference_number']

all_documents = load_data(DATA_PATH, text_col, metadata_cols)
len(all_documents)

167

#Evaluate ideal chunk size for the RAG system

In [4]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.core.evaluation import (
    DatasetGenerator,
    FaithfulnessEvaluator,
    RelevancyEvaluator
)
from llama_index.llms.openai import OpenAI

import openai
import os
import time

from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')

Generate questions to be used for query

In [10]:
# To evaluate for each chunk size, generate 10 questions from the input documents
eval_documents = all_documents
data_generator = DatasetGenerator.from_documents(eval_documents)
eval_questions = data_generator.generate_questions_from_nodes(num = 10)

/usr/local/lib/python3.10/dist-packages/llama_index/core/evaluation/dataset_generation.py:200: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(
/usr/local/lib/python3.10/dist-packages/llama_index/core/evaluation/dataset_generation.py:296: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [11]:
eval_questions

['What is the title of the regulatory act mentioned in the context information?',
 'When was the Tea Subsidy Act, No. 12 of 1958 issued?',
 'Who is the issuing authority of the Tea Subsidy Act?',
 'What is the main purpose of the Tea Subsidy Act?',
 'What is the reference number of the Tea Subsidy Act?',
 'What does the Tea Subsidy Act aim to subsidize?',
 'What is the significance of establishing a fund as mentioned in the Tea Subsidy Act?',
 'What is the export duty imposed by the Tea Subsidy Act?',
 'How does the Tea Subsidy Act support the replanting and rehabilitation of tea estates and small holdings?',
 'How does the Tea Subsidy Act contribute to the marketing and manufacturing of tea products?']

Initialization for Evaluation

In [12]:
# GPT-4 for evaluating the responses
gpt4 = OpenAI(temperature=0, model="gpt-4")

from llama_index.core import Settings
Settings.llm = gpt4

# Faithfulness and Relevancy Evaluators which are based on GPT-4
faithfulness_gpt4 = FaithfulnessEvaluator()
relevancy_gpt4 = RelevancyEvaluator()


Define function to evaluate metrics like response time, faithfulness and relevancy for a given chunk size

In [13]:
# Define function to calculate average response time, average faithfulness and average relevancy metrics for given chunk size
# GPT-3.5-Turbo to generate response and GPT-4 to evaluate it.
def evaluate_response_time_faithfulness_relevancy(chunk_size, eval_questions):

    total_response_time = 0
    total_faithfulness = 0
    total_relevancy = 0

    # create vector index
    llm = OpenAI(model="gpt-3.5-turbo")

    Settings.chunk_size = chunk_size
    Settings.llm = llm
    Settings.api_key = openai.api_key

    vector_index = VectorStoreIndex.from_documents(eval_documents)

    # By default, similarity_top_k is set to 2. To experiment with different values, pass it as an argument to as_query_engine()
    query_engine = vector_index.as_query_engine()
    num_questions = len(eval_questions)

    # Iterate over each question in eval_questions to compute metrics.
    for question in eval_questions:
        start_time = time.time()
        response_vector = query_engine.query(question)
        elapsed_time = time.time() - start_time

        faithfulness_result = faithfulness_gpt4.evaluate_response(
            response=response_vector
        ).passing

        relevancy_result = relevancy_gpt4.evaluate_response(
            query=question, response=response_vector
        ).passing

        total_response_time += elapsed_time
        total_faithfulness += faithfulness_result
        total_relevancy += relevancy_result

    average_response_time = total_response_time / num_questions
    average_faithfulness = total_faithfulness / num_questions
    average_relevancy = total_relevancy / num_questions

    return average_response_time, average_faithfulness, average_relevancy

Iterate over different chunk sizes to evaluate the metrics

In [15]:
# Iterate over different chunk sizes to evaluate the metrics

for chunk_size in [256, 512, 1024, 2048]:
  avg_response_time, avg_faithfulness, avg_relevancy = evaluate_response_time_faithfulness_relevancy(chunk_size, eval_questions)
  print(f"Chunk size {chunk_size} - Average Response time: {avg_response_time:.2f}s, Average Faithfulness: {avg_faithfulness:.2f}, Average Relevancy: {avg_relevancy:.2f}")

Chunk size 256 - Average Response time: 2.88s, Average Faithfulness: 0.80, Average Relevancy: 0.80
Chunk size 512 - Average Response time: 1.31s, Average Faithfulness: 1.00, Average Relevancy: 1.00
Chunk size 1024 - Average Response time: 1.05s, Average Faithfulness: 0.90, Average Relevancy: 0.90
Chunk size 2048 - Average Response time: 1.19s, Average Faithfulness: 0.90, Average Relevancy: 0.90
